In [ ]:
def simple_tokenizer(sentence):
  return sentence.split()

In [ ]:
!pip install paddlepaddle --pre paddlenlp

In [ ]:
sequence = "Let's have a tea break in CityU canteen tomorrow 3pm?"
tokens = simple_tokenizer(sequence)

print(tokens)

["Let's", 'have', 'a', 'tea', 'break', 'in', 'CityU', 'canteen', 'tomorrow', '3pm?']


In [ ]:
from transformers import AutoTokenizer

tokenizer_en = AutoTokenizer.from_pretrained('bert-base-cased')
text = "Let's have a tea break in CityU canteen tomorrow 3pm?"

tokens_en = tokenizer_en.tokenize(text)
print(tokens_en)
token_ids_en = tokenizer_en.convert_tokens_to_ids(tokens)
print(token_ids_en)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['Let', "'", 's', 'have', 'a', 'tea', 'break', 'in', 'City', '##U', 'can', '##teen', 'tomorrow', '3', '##pm', '?']
[100, 1138, 170, 5679, 2549, 1107, 100, 100, 4911, 100]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


https://paddlenlp.readthedocs.io/zh/latest/source/paddlenlp.transformers.roformer.modeling.html#paddlenlp.transformers.roformer.modeling.RoFormerPretrainedModel

In [ ]:
from paddlenlp.transformers import RoFormerTokenizer

chinese_text = '明天3时半到城市大学餐厅下午茶好吗？'
tokenizer_zh = RoFormerTokenizer.from_pretrained('roformer-chinese-base')
tokens_zh = tokenizer_zh.tokenize(chinese_text)
print(tokens_zh)
token_ids_zh = tokenizer_zh.convert_tokens_to_ids(tokens_zh)
print(token_ids_zh)

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[2024-10-04 08:10:17,364] [ WARNING] - if you run ring_flash_attention.py, please ensure you install the paddlenlp_ops by following the instructions provided at https://github.com/PaddlePaddle/PaddleNLP/blob/develop/csrc/README.md
[2024-10-04 08:10:22,001] [    INFO] - tokenizer config file saved in /root/.paddlenlp/models/roformer-chinese-base/tokenizer_config.json
[2024-10-04 08:10:22,008] [    INFO] - Special tokens file saved in /root/.paddlenlp/models/roformer-chinese-base/special_tokens_map.json
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tm

['明天', '3', '时', '半', '到', '城市', '大学', '餐厅', '下午茶', '好', '吗', '？']
[20898, 124, 2251, 702, 615, 14056, 14638, 34407, 36297, 1266, 815, 5665]


In [ ]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel
from paddlenlp.transformers import RoFormerModel, RoFormerTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Return a tensor of token_ids instead of a list of words
# Tokenization for English sentence
tokens_en = tokenizer_en(text, return_tensors="pt", padding=True, truncation=True)

# Tokenization for Chinese sentence
tokens_zh = tokenizer_zh(chinese_text, return_tensors="pd", padding=True, truncation=True)



In [ ]:
# Model for the English sentence
model_en = BertModel.from_pretrained('bert-base-uncased')

# Model for the Chinese sentence
model_zh = RoFormerModel.from_pretrained('roformer-chinese-base')#'bert-base-chinese')

[2024-10-04 08:10:28,107] [    INFO] - Loading weights file from cache at /root/.paddlenlp/models/roformer-chinese-base/model_state.pdparams
[2024-10-04 08:10:31,235] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-10-04 08:10:45,640] [ WARNING] - Some weights of the model checkpoint at roformer-chinese-base were not used when initializing RoFormerModel: ['cls.predictions.decoder_bias', 'cls.predictions.decoder_weight', 'cls.predictions.layer_norm.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing RoFormerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RoFormerModel from the checkpoint of a model that you expect to be 

In [ ]:
# Get embeddings for the English sentence
with torch.no_grad():
    embeddings_text = model_en(**tokens_en).last_hidden_state.mean(dim=1)

# Get embeddings for the Chinese sentence
with torch.no_grad():
    embeddings_chinese = model_zh(**tokens_zh)[0].mean(axis=1)




In [ ]:
# Measure cosine similarity between the two embeddings
similarity = cosine_similarity(embeddings_text.numpy(), embeddings_chinese.numpy())[0][0]
print(f"Similarity between English and Chinese sentence: {similarity}")

# Additional sentence with different meaning for comparison
different_sentence = "have you had lunch yet?"#"The weather is sunny today."

# Tokenization for the different sentence (using English tokenizer)
tokens_different = tokenizer_en(different_sentence, return_tensors="pt", padding=True, truncation=True)

# Get embeddings for the different sentence
with torch.no_grad():
    embeddings_different = model_en(**tokens_different).last_hidden_state.mean(dim=1)

# Measure similarity between the original English sentence and the different one
similarity_different = cosine_similarity(embeddings_text.numpy(), embeddings_different.numpy())[0][0]
print(f"Similarity between original English sentence and different sentence: {similarity_different}")

Similarity between English and Chinese sentence: 0.0062678661197423935
Similarity between original English sentence and different sentence: 0.7358040809631348
